In [1]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import layers
import numpy as np

In [2]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']
# positive, negative한 문장이 섞여있음
# -> 긍정, 부정으로 target을 정함
targets = [[1], [0], [1], [1], [0], [1]]   # target을 수동으로 맞춰줌

In [3]:
tokenizer = preprocessing.text.Tokenizer()   # keras의preprocessing으로 Tokenizer를 만듬
tokenizer.fit_on_texts(samples)   # 여기에 samples 넣음

In [4]:
print(tokenizer.word_index)   # tokenizer에 word_index 찍어보면  ->  단어 사전(dict) 를 보여줌  -> 오늘은 1번, 좋은은 2번

{'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}


In [5]:
# 저 문장을 실제 index로 바꿔주는 것
# 입력 단어 data를 index로 바꿔줌
sequences = tokenizer.texts_to_sequences(samples)
print(sequences)

# 결국 sequences <- 얘가 입력 데이터가 됨   /  위의 targets <- 이놈이 target이고

[[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]


In [6]:
input_sequences = np.array(sequences)
labels = np.array(targets)
# 입력과 타겟을 다 numpy배열로 바꾼 것

In [7]:
word_index = tokenizer.word_index
# 위의 word_index를 이 word_index에 담아둠

In [8]:
print(input_sequences)

[[ 4  1  5  6]
 [ 7  1  8  9]
 [10  2  3 11]
 [12  2  3 13]
 [14  1 15 16]
 [17 18 19 20]]


In [9]:
print(labels)

[[1]
 [0]
 [1]
 [1]
 [0]
 [1]]


In [10]:
# 위 둘 다 numpy 배열로 바꾼것 출력

In [11]:
# 필요한 변수 선언
batch_size = 2
num_epochs = 100

In [12]:
# 단어 사이즈
vocab_size = len(word_index) + 1    # +1 한 이유
emb_size = 128
hidden_dimension = 256
output_dimension = 1

# hidden_dimension, output_dimension 는 -> Subclassing (Custom Model)의 함수 예제의 변수

In [13]:
# 이걸 우리가 쓰던 Sequential 형태로 바꿔보자
model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length=4))  
# vocal_size / 몇차원으로 임베딩 할거냐/ 인풋 렝스는 4로 위에서 고정되어있음
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis=1)))   # 차원을 하나 줄여주는 역할 -> flatten하고 비슷(안에 있는 데이터는 변형 시키지 않고 찌부러뜨려서 차원을 줄임)
# reduce_mean은 평균값을 구한 다음 찌부러트려서 차원을 하나 줄임.  그점이 flatten과 차이점
model.add(layers.Dense(hidden_dimension, activation='relu'))
model.add(layers.Dense(output_dimension, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),     # 이런식으로 함수를 직접 쓸 수 있음. -> tf밑에 keras밑에 optimaizers밑에 Adam으로 0.001 넣어줌
             metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            2688      
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [15]:
# output shape의 (None, 4, 128)  <- 출력이 이렇게 나옴

# tf.reduce_mean  ->  특정 축으로 평균값을 구함 -> 차원이 하나 줄음
# aixs=1 이면 -> 가운데 , 즉 4라는 소리.
# 
# 4,1,5,6이 들어오면 128차원으로 바꿔줌. 그래서 (..4, 128) -> 128이 나옴

In [16]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)  # 위에 미리 변수 num_epochs,batch_size 만들어둠
# 여기까지가 p40의 내용

Epoch 1/100
3/3 [==============================] - 1s 7ms/step - loss: 0.6884 - accuracy: 0.8333
Epoch 2/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6666 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6487 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 8ms/step - loss: 0.6311 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 7ms/step - loss: 0.6078 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5853 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 7ms/step - loss: 0.5566 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 8ms/step - loss: 0.5254 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 8ms/step - loss: 0.4883 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 7ms/step - loss: 0.4494 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

In [ ]:
# 동일한 방법을 subclassing 방법으로 해보겠음

In [18]:
class CustomModel(tf.keras.Model):
    # 반드시 만들어야한 2개중 1개 = __init__ 함수
    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')  # class이름을 1번째 인자로 주고, self 주고, .__init__을 호출, 그리고 이름을 자기가 정해서 줌
        #우린 embedding, dense layer 2개
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu') #출력은 hidden_dimension으로 잡음
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')
        
    # 실제로 호출되서 실행되는 부분 -> call 함수
    def call(self, inputs):
        x = self.embedding(inputs)  # self.embedding에 입력onputs을 집어넣는다
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)  # 출력 x를 담음
        x = self.output_layer(x)
        # 이러면 출력 결과가 x에 담기고
        
        # 출력한곳에 x를 반환(return)해주면 됨
        return x

In [20]:
# model = custom model 객체를 만들어 주면 됨
model = CustomModel(vocab_size=vocab_size,
                   embed_dimension=emb_size,
                   hidden_dimension=hidden_dimension,
                   output_dimension=output_dimension)
# 왼쪽 변수들은 위의 class CustomModel의 __init 안의 변수들,
# 오른쪽의 수들은 위의
# # 이걸 우리가 쓰던 Sequential 형태로 바꿔보자
# model = tf.keras.Sequential()
# model.add(layers.Embedding(vocab_size, emb_size, ...
# model.add(layers.Dense(hidden_dimension, activation='relu'))
# model.add(layers.Dense(output_dimension, activation='sigmoid'))
# 여기서 가져옴
# 여기서 가져와서 하나씩 넣어주면 됨

model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

# 이 model.compile과, model.fit~~~은
# 위에 있는것 그대로 가져왔음..
# sequence->subclass 예제이므로, 사용법 익히는중이라, 같은 부분인듯.

Epoch 1/100
3/3 [==============================] - 1s 6ms/step - loss: 0.6894 - accuracy: 0.8333
Epoch 2/100
3/3 [==============================] - 0s 14ms/step - loss: 0.6695 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 39ms/step - loss: 0.6534 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 40ms/step - loss: 0.6371 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 56ms/step - loss: 0.6170 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 56ms/step - loss: 0.5928 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 42ms/step - loss: 0.5676 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 40ms/step - loss: 0.5358 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 53ms/step - loss: 0.4980 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 40ms/step - loss: 0.4595 - accuracy: 1.0000
Epoch 11/100
3/3 [==

In [ ]:
# 자연어 처리 입문 참고 추천
# https://wikidocs.net/book/2155